In [1]:
import influxdb_client
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
from datetime import datetime

In [2]:
# Set InfluxDB connection details
url = "http://localhost:8086"  # Replace with your InfluxDB URL
token = "c4hEuDHM9LPxNfWH-oFyFfOeIDihTnQx_OEeg99mrd6bZFNa1hEJx7a_aV-cFUH5nWquXUWU5IAeQ975itS4MQ=="  # Replace with your InfluxDB token
org = "trade_smart"  # Replace with your InfluxDB organization
bucket = "trade_smart"  # Replace with your InfluxDB bucket

# Instantiate the InfluxDB client
client = InfluxDBClient(url=url, token=token)

In [7]:
# Create the write API client
# Set InfluxDB connection details
url = "http://localhost:8086"  # Replace with your InfluxDB URL
token = "c4hEuDHM9LPxNfWH-oFyFfOeIDihTnQx_OEeg99mrd6bZFNa1hEJx7a_aV-cFUH5nWquXUWU5IAeQ975itS4MQ=="  # Replace with your InfluxDB token
org = "trade_smart"  # Replace with your InfluxDB organization
bucket = "trade_smart"  # Replace with your InfluxDB bucket
measurement = 'candlestick'
# Instantiate the InfluxDB client
client = InfluxDBClient(url=url, token=token)
write_api = client.write_api(write_options=SYNCHRONOUS)

In [126]:


data = [{
    "measurement": "candlestick",
    "tags": {"symbol": "test"},
    "time": 3,
    "fields": 
        {"open": 150.0, "high": 155.0, "low": 145.0, "close": 148.0, "adj_close": 148.0, "volume": 1000000}
    }
]
df = pd.DataFrame(data)


In [127]:
df

,measurement,tags,time,fields
0,candlestick,{'symbol': 'test'},3,"{'open': 150.0, 'high': 155.0, 'low': 145.0, '..."


In [128]:
for _, row in df.iterrows():
    print(row["fields"])


{'open': 150.0, 'high': 155.0, 'low': 145.0, 'close': 148.0, 'adj_close': 148.0, 'volume': 1000000}


In [129]:
# Point.from_dict({
#     "measurement": "candlestick",
#     "tags": {"symbol": "test"},
#     "time": 1,
#     "fields": 
#         {"open": 150.0, "high": 155.0, "low": 145.0, "close": 148.0, "Adj Close": 148.0, "volume": 1000000}
#     })
# points = []
# for _d in data:
#     points.append(Point.from_dict(_d))

# # Convert DataFrame to InfluxDB line protocol format
points = []
for row in data:
    point = Point("candlestick").tag("symbol", "test").field("open", row["fields"]["open"]).field("high", row["fields"]["high"]).field("low", row["fields"]["low"]).field("close", row["fields"]["close"]).field("adj_close", row["fields"]["adj_close"]).time(row["time"])
    points.append(point)
# [p for p in points]
line_protocol = "\n".join([p.to_line_protocol() for p in points])


In [130]:
line_protocol

'candlestick,symbol=test adj_close=148,close=148,high=155,low=145,open=150 3'

In [131]:

# Write data to InfluxDB
write_api.write(bucket=bucket, org=org, record=line_protocol)


In [106]:
query = f'from(bucket: "{bucket}") |> range(start: 0) |> filter(fn: (r) => r._measurement == "{measurement}")'
delete_query = f'from(bucket: "{bucket}") |> range(start: 0) |> filter(fn: (r) => r._measurement == "{measurement}") |> delete()'
start = datetime.utcfromtimestamp(0)  # Unix epoch start time (1970-01-01 00:00:00 UTC)
end = datetime.utcnow() 
# tables = client.delete_api().delete(start=start, stop=end, predicate = f'_measurement="{measurement}"', bucket=bucket, org=org)

flux_result = client.query_api().query(query, org=org)


In [132]:
for table in flux_result:
    for record in table.records:
        print(record.values)

In [108]:
df = pd.DataFrame(flux_result)
df_pivot = df.pivot(index='_time', columns='_field', values='_value')
df_pivot

KeyError: '_time'

In [133]:
# Close the connection
client.close()

In [146]:
import asyncio

In [147]:
loop = asyncio.get_event_loop()

In [148]:
loop.stop()
loop.run_forever()

/opt/homebrew/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'execute_parallel' was never awaited
  return compile(source, filename, mode, flags,


RuntimeError: This event loop is already running

In [1]:
import asyncio
import time
async def print_hello(input):
    time.sleep(1)
    print(f'Hello : {input}')
    
async def using_task_group():
    async with asyncio.TaskGroup() as tg:
        for i in range(10):
            tg.create_task(print_hello(i))
    print("All tasks have completed now.")
    
async def using_gather():
    result = await asyncio.gather(*[print_hello(i) for i in range(10)], return_exceptions=True)
    
async def main():
    from datetime import datetime
    start = datetime.now()
#     await using_task_group()
    await using_gather()
    end = datetime.now()
    td = (end - start).total_seconds() * 10**3
    print(f"Time taken: {td} ms")

# loop = asyncio.get_event_loop()
# loop.create_task(main())


In [2]:
print(f"The time of execution of above program is : {td:.03f}ms")

NameError: name 'td' is not defined

In [6]:
def print_hello(input):
    time.sleep(1)
    print(f"Hello : {input}")

from datetime import datetime
start = datetime.now()
for i in range(10):
    print_hello(i)
end = datetime.now()
td = (end - start).total_seconds() * 10**3
print(f"The time of execution of above program is : {td:.03f}ms")

Hello : 0
Hello : 1
Hello : 2
Hello : 3
Hello : 4
Hello : 5
Hello : 6
Hello : 7
Hello : 8
Hello : 9
The time of execution of above program is : 10036.617ms


In [4]:
print(f"The time of execution of above program is : {td:.03f}ms")

The time of execution of above program is : 172.892ms


In [143]:
await using_gather()

NameError: name 'using_gather' is not defined

In [5]:
import asyncio

async def my_function():
    # Simulate a function with a runtime of 1 second
    await asyncio.sleep(10)
    print("Function executed.")
class abc:
    async def execute_parallel(self):
        tasks = []
        for _ in range(10):
            task = asyncio.create_task(my_function())
            tasks.append(task)
        await asyncio.gather(*tasks)

await abc().execute_parallel()
print("All functions have completed.")

Function executed.
Function executed.
Function executed.
Function executed.
Function executed.
Function executed.
Function executed.
Function executed.
Function executed.
Function executed.
All functions have completed.


In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
#Data Source
import yfinance as yf

msft = yf.Ticker("IDFCFIRSTB.NS")
# get historical market data
hist = msft.history(period="1d", interval="1m",
                start=None, end=None, prepost=False, actions=True,
                auto_adjust=True, back_adjust=False, repair=True, keepna=False,
                proxy=None, rounding=False, timeout=10,
                raise_errors=False)
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Repaired?
Datetime,,,,,,,,
2023-07-07 09:15:00+05:30,80.449997,80.449997,80.000000,80.250000,0,0.0,0.0,NaN
2023-07-07 09:16:00+05:30,80.300003,80.300003,80.099998,80.250000,293733,0.0,0.0,NaN
2023-07-07 09:17:00+05:30,80.199997,80.300003,80.099998,80.300003,238919,0.0,0.0,NaN
2023-07-07 09:18:00+05:30,80.300003,80.449997,80.199997,80.400002,213057,0.0,0.0,NaN
2023-07-07 09:19:00+05:30,80.400002,80.449997,80.300003,80.300003,171577,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
2023-07-07 15:25:00+05:30,79.650002,79.800003,79.550003,79.800003,242367,0.0,0.0,NaN
2023-07-07 15:26:00+05:30,79.750000,79.750000,79.650002,79.750000,110695,0.0,0.0,NaN
2023-07-07 15:27:00+05:30,79.750000,79.750000,79.699997,79.699997,99780,0.0,0.0,NaN


In [4]:
import influxdb_client
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
from datetime import datetime

def influx_function(measurement = None, tag_dict = None, fields_dataframe = None):
    if measurement is None or tag_dict is None or fields_dataframe is None:
        return {"success": False, "error": "Incorrect input Arguments"}

    # Create the write API client
    # Set InfluxDB connection details
    url = "http://localhost:8086"  # Replace with your InfluxDB URL
    token = "c4hEuDHM9LPxNfWH-oFyFfOeIDihTnQx_OEeg99mrd6bZFNa1hEJx7a_aV-cFUH5nWquXUWU5IAeQ975itS4MQ=="  # Replace with your InfluxDB token
    org = "trade_smart"  # Replace with your InfluxDB organization
    bucket = "trade_smart"  # Replace with your InfluxDB bucket
    
    # Instantiate the InfluxDB client
    client = InfluxDBClient(url=url, token=token)
    write_api = client.write_api(write_options=SYNCHRONOUS)
    
    # # Convert DataFrame to InfluxDB line protocol format
    points = []
    for _, row in fields_dataframe.iterrows():
        point = Point("candlestick")
        for tag_key, tag_value in tag_dict.items():
            point.tag(tag_key, tag_value)
            
        for column, value in row.items():
            point.field(column, value)
#         point.field("open", row["Open"])
#         point.field("high", row["High"])
#         point.field("low", row["Low"])
#         point.field("close", row["Close"])
#         point.field("volume", row["Volume"])
        
        point.time(_)
        points.append(point)
    
    # [p for p in points]
    line_protocol = "\n".join([p.to_line_protocol() for p in points])
    return line_protocol
    # Write data to InfluxDB
#     write_api.write(bucket=bucket, org=org, record=line_protocol)

    # Close the connection
#     client.close()
    
def fetch_records():
    client = InfluxDBClient(url=url, token=token)
    query = f'from(bucket: "{bucket}") |> range(start: 0) |> filter(fn: (r) => r._measurement == "{measurement}")'
    tables = client.query_api().query(query, org=org)
    return tables
    client.close()

def delete_records():
    client = InfluxDBClient(url=url, token=token)
    delete_query = f'from(bucket: "{bucket}") |> range(start: 0) |> filter(fn: (r) => r._measurement == "{measurement}") |> delete()'
    start = datetime.utcfromtimestamp(0)  # Unix epoch start time (1970-01-01 00:00:00 UTC)
    end = datetime.utcnow() 
    tables = client.delete_api().delete(start=start, stop=end, predicate = f'_measurement="{measurement}"', bucket=bucket, org=org)
    client.close()

In [5]:
x = influx_function(measurement = 'candlestick', tag_dict = {'symbol': 'test_idfc'}, fields_dataframe = hist)
len(x.split("\n"))

NameError: name 'hist' is not defined

In [60]:
for i in range(0, len(hist), 100):
    chunk = hist.iloc[i:i+100]
    processed_chunk = influx_function(measurement = 'candlestick', tag_dict = {'symbol': 'test_idfc'}, fields_dataframe = chunk)
    records = fetch_records()


100
Processed chunk:
100
100
Processed chunk:
100
100
Processed chunk:
100
75
Processed chunk:
75


In [103]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Repaired?
Datetime,,,,,,,,
2023-07-07 09:15:00+05:30,80.449997,80.449997,80.000000,80.250000,0,0.0,0.0,NaN
2023-07-07 09:16:00+05:30,80.300003,80.300003,80.099998,80.250000,293733,0.0,0.0,NaN
2023-07-07 09:17:00+05:30,80.199997,80.300003,80.099998,80.300003,238919,0.0,0.0,NaN
2023-07-07 09:18:00+05:30,80.300003,80.449997,80.199997,80.400002,213057,0.0,0.0,NaN
2023-07-07 09:19:00+05:30,80.400002,80.449997,80.300003,80.300003,171577,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
2023-07-07 15:25:00+05:30,79.650002,79.800003,79.550003,79.800003,242367,0.0,0.0,NaN
2023-07-07 15:26:00+05:30,79.750000,79.750000,79.650002,79.750000,110695,0.0,0.0,NaN
2023-07-07 15:27:00+05:30,79.750000,79.750000,79.699997,79.699997,99780,0.0,0.0,NaN


In [137]:
flux_result = fetch_records()

In [138]:

# Extract time, field, and value from the Flux result
time_values = []
field_values = []
value_values = []

for item in flux_result:
    for record in item:
        time_values.append(record.values['_time'])
        field_values.append(record.values['_field'])
        value_values.append(record.values['_value'])

# Create a DataFrame from the extracted values
df = pd.DataFrame({'_time': time_values, '_field': field_values, '_value': value_values})

# Pivot DataFrame to have fields as columns
df_pivot = df.pivot(index='_time', columns='_field', values='_value')

print(df_pivot)

_field                         close       high        low       open  \
_time                                                                   
2023-07-07 03:45:00+00:00  80.250000  80.449997  80.000000  80.449997   
2023-07-07 03:46:00+00:00  80.250000  80.300003  80.099998  80.300003   
2023-07-07 03:47:00+00:00  80.300003  80.300003  80.099998  80.199997   
2023-07-07 03:48:00+00:00  80.400002  80.449997  80.199997  80.300003   
2023-07-07 03:49:00+00:00  80.300003  80.449997  80.300003  80.400002   
...                              ...        ...        ...        ...   
2023-07-07 09:55:00+00:00  79.800003  79.800003  79.550003  79.650002   
2023-07-07 09:56:00+00:00  79.750000  79.750000  79.650002  79.750000   
2023-07-07 09:57:00+00:00  79.699997  79.750000  79.699997  79.750000   
2023-07-07 09:58:00+00:00  79.699997  79.750000  79.650002  79.699997   
2023-07-07 09:59:00+00:00  79.699997  79.900002  79.699997  79.699997   

_field                       volume  
_time       

In [139]:
df_pivot.describe()

_field,close,high,low,open,volume
count,375.000000,375.000000,375.000000,375.000000,375.000000
mean,79.683734,79.711600,79.637467,79.683867,72613.368000
std,0.376111,0.381497,0.369133,0.377297,67411.473535
min,79.050003,79.099998,79.050003,79.050003,0.000000
25%,79.400002,79.449997,79.400002,79.400002,26478.500000
50%,79.599998,79.599998,79.550003,79.599998,50721.000000
75%,79.900002,79.949997,79.849998,79.900002,94123.500000
max,80.750000,80.800003,80.699997,80.750000,423819.000000


In [9]:
x = fetch_records()

In [ ]:
for i in x:
    for j in i:
        print(